In [199]:
%matplotlib inline

In [200]:
from helpers import parse_animetas_data, parse_sale_data, parse_listing_data

import requests
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import pickle

from statistics import *
from scipy.stats import pearsonr, chi2_contingency

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import time
from datetime import date, timedelta, datetime

import os
import glob

#### Getting Animetas Assets Data

The code below collects assets data about the 10,101 Animetas. The API has a limit of 50 items per call, and therefore we need to create a loop with 204 iterations to collect all Animetas data.

##### Animeta Data Parsed:
- 'animeta_id'
- 'creator_username'
- 'creator_address'
- 'owner_username'
- 'owner_address'
- 'traits'
- 'num_sales'

In [201]:
def download_animetas_info(save_location):
    url = "https://api.opensea.io/api/v1/assets"
    listometas = []

    for i in range(0, 204):
        querystring = {"token_ids":list(range((i*50), (i*50)+50)),
                       "asset_contract_address":"0x18df6c571f6fe9283b87f910e41dc5c8b77b7da5",
                       "order_direction":"desc",
                       "offset":"0",
                       "limit":"50"}
        response = requests.request("GET", url, params=querystring)

        print(i, end=" ")
        if response.status_code != 200:
            print('error')
            break

        # Getting animetas data
        animetas = response.json()['assets']
        # Parsing animetas data
        parsed_animetas = [parse_animetas_data(animeta) for animeta in animetas]
        # storing parsed animetas data into list
        listometas.append(parsed_animetas)
    
    # Flatten everything into one list
    listometas = [item for sublist in listometas for item in sublist]
    # Convert to df
    animetas_df = pd.DataFrame(listometas)
    
    with open(save_location + 'animetas_df'+str(date.today())+r'.pkl', 'wb') as f:
        pickle.dump(animetas_df, f)

#### Getting animetas sales transactions data

The code below collects all sale transactions data. 
The API has a limit of 300 items per call and 10,000 items total / response. 
Days with > 10,000 sales will require chunking (built in) but most projects aren't there yet.

##### Sales Data Parsed
- 'is_bundle'
- 'animeta_id'
- 'seller_address'
- 'buyer_address'
- 'buyer_username'
- 'seller_username'
- 'timestamp'
- 'total_price'
- 'payment_token'
- 'usd_price'
- 'transaction_hash'

In [202]:
# Download sales info from start_date to end _date and save them all into their own day's files
# Default values to first sale ever 7/30/21 to today
def download_sales_info(save_location, start_date = date(2021, 7, 30), end_date = date.today()):
    url = "https://api.opensea.io/api/v1/events"
    # get the number of days that we want to download and save sales for
    delta = end_date - start_date
    count_days = int(delta.days)
    
    for i in range(count_days+1):
        sales_that_day = []
        # set start and end of the day we are checking, if it's today set end to current time
        if date.today() == (start_date + timedelta(days=i)):
            before = datetime.now()
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        else:
            before = datetime.combine((start_date + timedelta(days=i+1)), datetime.min.time())
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        # There are too many transactions, now have to break them up by chunks in the day
        hour_chunks = 12
        chunk_count = 24/hour_chunks
        
        for chunk in range(int(chunk_count)):
            end = False
            for j in range(0, 35):
                # add the hour_chunk to the start of the day (after) time for each chunk
                # use the actual before if we pass it chronologically though
                changed_before = after + timedelta(hours=hour_chunks*(chunk+1)) - timedelta(minutes = 1)
                changed_after = after + timedelta(hours = hour_chunks*(chunk))
                
                # this should only happen on the last chunk of a split day or if on current day
                if before < changed_before:
                    changed_before = before
                    end = True

                querystring = {"asset_contract_address":"0x18Df6C571F6fE9283B87f910E41dc5c8b77b7da5",
                               "event_type":"successful",
                               "only_opensea":"true",
                               "offset":j*300,
                               "occurred_before":changed_before,
                               "occurred_after":changed_after,
                               "limit":"300"}
                headers = {"Accept": "application/json"}

                response = requests.request("GET", url, headers=headers, params=querystring)


                print(j, end=" ")
                if response.status_code != 200:
                    print('error')
                    print(response.json())
                    break

                #Getting animetas sales data
                animeta_sales = response.json()['asset_events']

                if animeta_sales == []:
                    keepgoing = False
                    break

                # Parsing animetas sales data
                parsed_animeta_sales = [parse_sale_data(sale) for sale in animeta_sales]
                # storing parsed animetas data into list
                sales_that_day.append(parsed_animeta_sales)
                # check if the last date in the list is the same day as 
                last_date = (datetime.strptime(parsed_animeta_sales[0]['timestamp'], '%Y-%m-%dT%H:%M:%S'))
                print(last_date)
            if end:
                break
        sales_that_day = [item for sublist in sales_that_day for item in sublist]
        
        print(str(len(sales_that_day))+ " sales saved to" + save_location + "animetas_sales_list_" + str(start_date + timedelta(days=i))+'.pkl')
        with open(save_location + "animetas_sales_list_" + str(start_date + timedelta(days=i))+'.pkl', 'wb') as f:
            pickle.dump(sales_that_day, f)


#### Getting animetas listings transactions data

The code below collects all listings of animetas on OpenSea. 
The API has a limit of 300 items per call and 10,000 items total / response. 
Days with > 10,000 listings will require chunking (built in).


##### Listing Data Parsed:
- 'is_bundle'
- 'animeta_id'
- 'seller_address'
- 'seller_username'
- 'created_date'
- 'starting_price'
- 'payment_token'
- 'usd_price'

In [203]:
# Download listings info from start_date to end _date and save them all into their own day's files
# Default values to first listing ever 7/30/21 to today
def download_listings_info(save_location, start_date = date(2021, 7, 30), end_date = date.today()):
    url = "https://api.opensea.io/api/v1/events"
    # get the number of days that we want to download and save listings for
    delta = end_date - start_date
    count_days = int(delta.days)
    
    for i in range(count_days+1):
        listings_that_day = []
        # set start and end of the day we are checking, if it's today set end to current time
        if date.today() == (start_date + timedelta(days=i)):
            before = datetime.now()
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        else:
            before = datetime.combine((start_date + timedelta(days=i+1)), datetime.min.time())
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        # There are too many transactions, now have to break them up by chunks in the day
        hour_chunks = 8
        chunk_count = 24/hour_chunks
        
        for chunk in range(int(chunk_count)):
            end = False
            for j in range(0, 35):
                # add the hour_chunk to the start of the day (after) time for each chunk
                # use the actual before if we pass it chronologically though
                changed_before = after + timedelta(hours=hour_chunks*(chunk+1)) - timedelta(minutes = 1)
                changed_after = after + timedelta(hours = hour_chunks*(chunk))
                
                # this should only happen on the last chunk of a split day or if on current day
                if before < changed_before:
                    changed_before = before
                    end = True

                querystring = {"asset_contract_address":"0x18Df6C571F6fE9283B87f910E41dc5c8b77b7da5",
                               "event_type":"created",
                               "only_opensea":"true",
                               "offset":j*300,
                               "occurred_before":changed_before,
                               "occurred_after":changed_after,
                               "limit":"300"}
                headers = {"Accept": "application/json"}

                response = requests.request("GET", url, headers=headers, params=querystring)


                print(j, end=" ")
                if response.status_code != 200:
                    print('error')
                    print(response.json())
                    break

                #Getting animetas listings data
                animeta_listings = response.json()['asset_events']

                if animeta_listings == []:
                    keepgoing = False
                    break

                # Parsing animetas listings data
                parsed_animeta_listings = [parse_listing_data(listing) for listing in animeta_listings]
                # storing parsed animetas data into list
                listings_that_day.append(parsed_animeta_listings)
                # check if the last date in the list is the same day as 
                print(parsed_animeta_listings[0]['created_date'])
            if end:
                break
        listings_that_day = [item for sublist in listings_that_day for item in sublist]
        
        print(str(len(listings_that_day))+ " listings saved to" + save_location +
              "animetas_listings_list_" + str(start_date + timedelta(days=i))+'.pkl')
        with open(save_location + "animetas_listings_list_" + str(start_date + timedelta(days=i))+'.pkl', 'wb') as f:
            pickle.dump(listings_that_day, f)

#### Each of the download files saves a pickled list / DF  of their information which can be loaded below for analysis
Sales and Listings downloads support start_date and end_date to avoid taking extra time to download old days of data. 

In [204]:
save_location = "./data/"

# ANIMETAS
#download_animetas_info(save_location)

# SALES
# download sales info from start_date to end_date e.g. date(2021, 7, 30), date.today() - timedelta(days=1), etc.
# defaults to first day of sales to today

download_sales_info(save_location = save_location, start_date = date.today())

# LISTINGS
# download listings info from start_date to end_date e.g. date(2021, 7, 30), date.today() - timedelta(days=1), etc.
# defaults to first day of listings to today

#download_listings_info(save_location = save_location, start_date = date.today())

0 2021-08-08 11:55:41
1 0 2021-08-08 15:50:54
1 154 sales saved to./data/animetas_sales_list_2021-08-08.pkl


#### Functions for loading data from pickled files

In [205]:
# load the sales lists, combine them, and turn into a DF
def load_sales_info(save_location):
    files = [filename for filename in os.listdir(save_location) if filename.startswith('animetas_sales')]
    all_sales = []
    # load all files for sales by day
    for file in files:
        with open(str(save_location) + str(file), 'rb') as f:
            all_sales.append(pickle.load(f))
    
    # flatten into one list
    all_sales = [item for sublist in all_sales for item in sublist]
    # convert to dataframe
    animetas_sales_df = pd.DataFrame(all_sales)
    
    return animetas_sales_df

# load the listing lists, combine them, and turn into a DF
def load_listings_info(save_location):
    files = [filename for filename in os.listdir(save_location) if filename.startswith('animetas_listing')]
    all_listings = []
    # load all files for listings by day
    for file in files:
        with open(str(save_location) + str(file), 'rb') as f:
            all_listings.append(pickle.load(f))
    
    # flatten into one list
    all_listings = [item for sublist in all_listings for item in sublist]
    # convert to dataframe
    animetas_listings_df = pd.DataFrame(all_listings)
    
    return animetas_listings_df

# load most recent saved animetas df
def load_animetas_info(save_location):
    files = glob.glob(str(save_location)+'animetas_df????-??-??.pkl')
    with open(max(files, key=os.path.getctime), 'rb') as f:
        return pickle.load(f)

#### Creating, filling, saving, and loading the dictionaries of traits and their respective sales

In [206]:
# make arrays for all of the traits e.g. eye type, weapon type, etc. and values e.g. blue, submachine gun, etc.
def create_trait_value_dicts(animetas_df):
    all_traits = []
    all_values = []
    for meta in animetas_df['traits']:
        for trait in meta:
            if trait['value'] not in all_values:
                all_values.append(trait['value'])
            if trait['trait_type'] not in all_traits:
                all_traits.append(trait['trait_type'])
    # create dictionaries to keep track of all sales for each trait
    sales_values_dict = dict.fromkeys(all_values,pd.DataFrame(columns=animetas_sales_df.columns))
    # create dictionaries to keep track of all listings for each trait
    listings_values_dict = dict.fromkeys(all_values,pd.DataFrame(columns=animetas_listings_df.columns))
    
    return sales_values_dict,listings_values_dict

# enhance the dictionary with avg sales, min sale, max sale, std dev of sales

# fill dictionary of every trait value with all sales from for that value
def fill_sales_values_dict(sales_values_dict,animetas_sales_df,animetas_df, save_location = None):
    # Time consuming to access value in dictionary in list in dataframe, how to optimize?
    # Fill each sales values dictionary entry with a dataframe of all sales involving that trait 
    count = 0
    start_time = time.time()
    for index, row in animetas_sales_df.iterrows():
        if index % 1000 == 0:
            print(index,len(animetas_sales_df['animeta_id']))
            print("--- %s seconds ---" % (time.time() - start_time))
        traits = animetas_df.loc[animetas_df['animeta_id'] == row['animeta_id']]['traits'].values[0]
        for trait in traits:
            sales_values_dict[trait['value']] = sales_values_dict[trait['value']].append(row)
    
    # save the dict to avoid repeating each time 
    if save_location:
        print("Dictionary saved to" + save_location + "sales_values_dict" + str(date.today())+'.pkl')
        with open(save_location + "sales_values_dict" + str(date.today())+'.pkl', 'wb') as f:
            pickle.dump(sales_values_dict, f)
        
    return sales_values_dict

# load most recent sales_values_dict
def load_sales_values_dict(save_location):
    files = glob.glob(str(save_location)+'sales_values_dict????-??-??.pkl')
    with open(max(files, key=os.path.getctime), 'rb') as f:
        return pickle.load(f)
    


#### Load and pre-process our saved files

In [207]:
# SALES
animetas_sales_df = load_sales_info(save_location)
# Pre-processing
# Convert price from WEI to ETH & for now get rid of bundles and duplicates(?)
animetas_sales_df = animetas_sales_df[(animetas_sales_df['payment_token'] != 'USDC') & (animetas_sales_df['is_bundle'] == False)].copy()
animetas_sales_df = animetas_sales_df.loc[animetas_sales_df.astype(str).drop_duplicates().index]
animetas_sales_df['total_price'] = animetas_sales_df['total_price']/10.**18
# Change timestamp to datetime
animetas_sales_df['timestamp'] = pd.to_datetime(animetas_sales_df['timestamp'])
# Calculating the sale prices in USD
#animetas_sales_df['total_price_usd'] = animetas_sales_df['total_price'] * animetas_sales_df['usd_price']
# Dropping columns not needed for analysis
animetas_sales_df = animetas_sales_df.drop(columns=['buyer_username', 'seller_username','transaction_hash'])

In [55]:
# LISTINGS
animetas_listings_df = load_listings_info(save_location)
# Pre-processing
# Convert price from WEI to ETH & for now get rid of bundles and duplicates(?)
animetas_listings_df = animetas_listings_df[(animetas_listings_df['payment_token'] != 'USDC') & (animetas_listings_df['is_bundle'] == False)].copy()
animetas_listings_df = animetas_listings_df.loc[animetas_listings_df.astype(str).drop_duplicates().index]
animetas_listings_df['starting_price'] = animetas_listings_df['starting_price']/10.**18
# Change timestamp to datetime
animetas_listings_df['created_date'] = pd.to_datetime(animetas_listings_df['created_date'])
# Calculating the sale prices in USD
#animetas_listings_df['total_price_usd'] = animetas_listings_df['starting_price'] * animetas_listings_df['usd_price']
# Dropping columns not needed for analysis
animetas_listings_df = animetas_listings_df.drop(columns=['seller_username'])

In [208]:
# ANIMETAS
animetas_df = load_animetas_info(save_location)
# Dropping columns not needed for analysis
animetas_df = animetas_df.drop(columns=['creator_username','creator_address','owner_username'])

In [211]:
# create the empty dictionary for each trait
#sales_values_dict,listings_values_dict = create_trait_value_dicts(animetas_df)

# time-consuming filling of dictionary that will give each trait all sales it has had
sales_values_dict = fill_sales_values_dict(sales_values_dict,animetas_sales_df,animetas_df, save_location = save_location)

# load the most recent saved sales_values_dict
sales_values_dict = load_sales_values_dict(save_location)

0 10773
--- 0.029035091400146484 seconds ---
1000 10773
--- 43.684308767318726 seconds ---
2000 10773
--- 86.8083815574646 seconds ---
3000 10773
--- 129.73203778266907 seconds ---
4000 10773
--- 173.31471300125122 seconds ---
5000 10773
--- 217.85678052902222 seconds ---
6000 10773
--- 267.37901520729065 seconds ---
7000 10773
--- 315.4133276939392 seconds ---
8000 10773
--- 372.1209976673126 seconds ---
9000 10773
--- 432.62146830558777 seconds ---
10000 10773
--- 491.1611559391022 seconds ---
Dictionary saved to./data/sales_values_dict2021-08-08.pkl


## Building our Dataset for Analysis

In [213]:
# build info for each animeta which can be merged with sales
# animeta_id, owner_address, num_sales, traits*
# * each trait will also have: avg sales, min sale, max sale, std dev of sales
def enhance_animetas(animetas_df, sales_values_dict, save_location = None):
    start_time = time.time()
    
    #make columns for each traits statistics
    for trait in sales_values_dict:
        animetas_df[str(trait)+"_mean"] = ""
        animetas_df[str(trait)+"_mean_today"] = ""
        animetas_df[str(trait)+"_min"] = ""
        animetas_df[str(trait)+"_min_today"] = ""
        animetas_df[str(trait)+"_max"] = ""
        animetas_df[str(trait)+"_max_today"] = ""
        animetas_df[str(trait)+"_stddev"] = ""
        animetas_df[str(trait)+"_stddev_today"] = ""
    
    
    yesterday = (date.today() - timedelta(days=1))
    yesterday = datetime(yesterday.year, yesterday.month, yesterday.day)
    # go through all rows
    for index, row in animetas_df.iterrows():
        if index % 1000 == 0:
            print(index,len(animetas_df['animeta_id']))
            print("--- %s seconds ---" % (time.time() - start_time))
        traits = row['traits']
        # for each trait fill columns created above
        for trait in traits:
            the_trait = trait['value']
            df = sales_values_dict[the_trait]
            
            if not df.empty:
                animetas_df.at[index,str(the_trait)+"_mean"] = df['total_price'].mean()
                animetas_df.at[index,str(the_trait)+"_min"] = min(df['total_price'])
                animetas_df.at[index,str(the_trait)+"_max"] = max(df['total_price'])

                if len(df) > 1:
                    animetas_df.at[index,str(the_trait)+"_stddev"] = stdev(df['total_price'])
                    
                
                if not df[df['timestamp'] >= yesterday].empty:
                    animetas_df.at[index,str(the_trait)+"_mean_today"] = (df[df['timestamp'] >= yesterday ]['total_price'].mean())
                    animetas_df.at[index,str(the_trait)+"_min_today"] = min(df[df['timestamp'] >= yesterday ]['total_price'])
                    animetas_df.at[index,str(the_trait)+"_max_today"] = min(df[df['timestamp'] >= yesterday ]['total_price'])
                    if len(df[df['timestamp'] >= yesterday ]) > 1:
                        animetas_df.at[index,str(the_trait)+"_max_today"] = stdev(df[df['timestamp'] >= yesterday ]['total_price'])


    
    if save_location:
        print("Enhanced Animetas DF" + save_location + "enhanced_animetas_df" + str(date.today())+'.pkl')
        with open(save_location + "enhanced_animetas_df" + str(date.today())+'.pkl', 'wb') as f:
            pickle.dump(animetas_df, f)
    
    return animetas_df

# load most recent enhanced animetas df
def load_enhanced_animetas(save_location):
    files = glob.glob(str(save_location)+'enhanced_animetas_df????-??-??.pkl')
    with open(max(files, key=os.path.getctime), 'rb') as f:
        return pickle.load(f)
            
    

# for each sale get the amount of ETH made or lost over previous sale
def fill_ETH_change(row, animetas_sales_df):
    # Get all sales for that animeta
    all_sales = animetas_sales_df[animetas_sales_df['animeta_id'] == row['animeta_id']]
    # Order from newest to oldest
    all_sales = all_sales.sort_values(by='timestamp', ascending=False)
    
    # If there wasn't at least one previous sale return 0, assuming we will buy after mint
    all_sales = all_sales[all_sales['timestamp'] <= row['timestamp']]
    if len(all_sales) < 2:
        return 0
    else:
        price_change = all_sales.iloc[0]['total_price'] - all_sales.iloc[1]['total_price']
        return price_change
    
    
    
    


In [214]:
print("The database has information about %d animetas." % len(animetas_df))
print("The database has information about %d animetas sale transactions." % len(animetas_sales_df))
print("The database has information about %d animetas listing transactions." % len(animetas_listings_df))

The database has information about 10101 animetas.
The database has information about 10773 animetas sale transactions.
The database has information about 35265 animetas listing transactions.


In [215]:
enhanced_animetas = enhance_animetas(animetas_df, sales_values_dict, save_location = save_location)

0 10101
--- 16.078285932540894 seconds ---
1000 10101
--- 194.16446733474731 seconds ---
2000 10101
--- 372.52961802482605 seconds ---
3000 10101
--- 555.615638256073 seconds ---
4000 10101
--- 733.5904068946838 seconds ---
5000 10101
--- 921.4437713623047 seconds ---
6000 10101
--- 1108.2606902122498 seconds ---
7000 10101
--- 1295.7344295978546 seconds ---
8000 10101
--- 1471.773710489273 seconds ---
9000 10101
--- 1645.9521193504333 seconds ---
10000 10101
--- 1820.1081385612488 seconds ---
Enhanced Animetas DF./data/enhanced_animetas_df2021-08-08.pkl


In [216]:
start_time = time.time()
# get change in price from last sale to this one
animetas_sales_df['ETH_delta_last_sale'] = animetas_sales_df.apply(lambda row: fill_ETH_change(row, animetas_sales_df), axis = 1) 
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.385906457901 seconds ---


In [234]:
combined_df['wool hat with earring_stddev_today'].empty

False

In [235]:
combined_df = pd.concat([animetas_sales_df, enhanced_animetas], axis=1, join='inner')
independent_var = combined_df.pop('ETH_delta_last_sale')
combined_df.insert(0, 'ETH_delta_last_sale', independent_var)
#combined_df.insert(6, 'date', pd.to_datetime(combined_df['timestamp']).dt.date)
combined_df.insert(6, 'year', combined_df['timestamp'].dt.year)
combined_df.insert(6, 'month', combined_df['timestamp'].dt.month)
combined_df.insert(6, 'day', combined_df['timestamp'].dt.day)
combined_df.insert(6, 'hour', combined_df['timestamp'].dt.hour)
combined_df.insert(10, 'weekday', combined_df['timestamp'].dt.dayofweek)
combined_df['seller_address'] = pd.factorize(combined_df['seller_address'])[0]
combined_df['buyer_address'] = pd.factorize(combined_df['buyer_address'])[0]
combined_df['owner_address'] = pd.factorize(combined_df['owner_address'])[0]
combined_df['is_bundle'] = pd.factorize(combined_df['is_bundle'])[0]
combined_df = combined_df.drop(columns=['traits','timestamp','payment_token'])
combined_df = combined_df.dropna(axis=0, how='all')
combined_df = combined_df.replace(r'^\s*$', np.nan, regex=True)

In [236]:
combined_df = combined_df.apply(pd.to_numeric, errors='raise')

In [237]:
combined_df.to_csv('combined_df.csv')

In [238]:
combined_df.head(5)

ETH_delta_last_sale  is_bundle  animeta_id  seller_address  buyer_address  \
0                  0.0          0          72               0              0   
1                  0.0          0         108               1              1   
2                  0.0          0         638               2              2   
3                  0.0          0         693               3              3   
4                  0.0          0         599               4              4   

   hour   day  month    year  weekday  total_price  usd_price  animeta_id  \
0  11.0  30.0    7.0  2021.0      4.0        0.400    3092.42          38   
1  11.0  30.0    7.0  2021.0      4.0        0.250    3092.42          35   
2  23.0  30.0    7.0  2021.0      4.0        0.295    3092.42          34   
3  23.0  30.0    7.0  2021.0      4.0        0.325    3092.42          48   
4  23.0  30.0    7.0  2021.0      4.0        0.310    3092.42          40   

   owner_address  num_sales  gamma-ray glasses_mean  \
0              0          0                0.442776   
1              1          0                     NaN   
2              2          0                     NaN   
3              3          0                     NaN   
4              4          1                     NaN   

   gamma-ray glasses_mean_today  gamma-ray glasses_min  \
0                      0.892722                  0.119   
1                           NaN                    NaN   
2                           NaN                    NaN   
3                           NaN                    NaN   
4                           NaN                    NaN   

   gamma-ray glasses_min_today  gamma-ray glasses_max  \
0                         0.65                   2.99   
1                          NaN                    NaN   
2                          NaN                    NaN   
3                          NaN                    NaN   
4                          NaN                    NaN   

   gamma-ray glasses_max_today  gamma-ray glasses_stddev  \
0                     0.156333                  0.371755   
1                          NaN                       NaN   
2                          NaN                       NaN   
3                          NaN                       NaN   
4                          NaN                       NaN   

   gamma-ray glasses_stddev_today  grin_mean  grin_mean_today  grin_min  \
0                             NaN   0.491571         0.934143      0.11   
1                             NaN        NaN              NaN       NaN   
2                             NaN        NaN              NaN       NaN   
3                             NaN        NaN              NaN       NaN   
4                             NaN        NaN              NaN       NaN   

   grin_min_today  grin_max  grin_max_today  grin_stddev  grin_stddev_today  \
0            0.83       4.2        0.116777     0.623297                NaN   
1             NaN       NaN             NaN          NaN                NaN   
2             NaN       NaN             NaN          NaN                NaN   
3             NaN       NaN             NaN          NaN                NaN   
4             NaN       NaN             NaN          NaN                NaN   

   human_mean  human_mean_today  human_min  human_min_today  human_max  \
0    0.403551           1.00846        0.0            0.098       11.5   
1    0.403551           1.00846        0.0            0.098       11.5   
2    0.403551           1.00846        0.0            0.098       11.5   
3    0.403551           1.00846        0.0            0.098       11.5   
4    0.403551           1.00846        0.0            0.098       11.5   

   human_max_today  human_stddev  human_stddev_today  submachine gun_mean  \
0         0.421569      0.346377                 NaN             0.808786   
1         0.421569      0.346377                 NaN                  NaN   
2         0.421569      0.346377                 NaN                  NaN

In [239]:
start_time = time.time()
corr = combined_df.corr()
print("--- %s seconds ---" % (time.time() - start_time))


--- 103.19627285003662 seconds ---


In [240]:
corr.to_csv('correlational_df.csv')